In [2]:
import pandas as pd
import numpy as np
import nltk

ImportError: No module named 'nltk'

# Baum-Welch Algorithm

In [238]:
alpha = np.zeros( (5, 9) )
alpha

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [18]:
X = np.reshape([i for i in range(9)]*5, (9,5))
X

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 0],
       [1, 2, 3, 4, 5],
       [6, 7, 8, 0, 1],
       [2, 3, 4, 5, 6],
       [7, 8, 0, 1, 2],
       [3, 4, 5, 6, 7],
       [8, 0, 1, 2, 3],
       [4, 5, 6, 7, 8]])

In [21]:
alpha.T[1] = X[1]
alpha

array([[ 0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [95]:
epsilon = np.random.random( (4, 3, 2) )
epsilon

array([[[ 0.98805003,  0.40372642],
        [ 0.53797724,  0.23448019],
        [ 0.05338353,  0.51783128]],

       [[ 0.86992223,  0.80669244],
        [ 0.65369358,  0.79343884],
        [ 0.6183715 ,  0.17904411]],

       [[ 0.8651411 ,  0.82390794],
        [ 0.32326851,  0.18333333],
        [ 0.24473372,  0.36866599]],

       [[ 0.60980616,  0.14317828],
        [ 0.82471555,  0.64424388],
        [ 0.43957701,  0.51154068]]])

In [96]:
for i in range(len(gamma)):
        sum_gamma = np.sum(gamma[i])
        gamma[i] = np.divide(gamma[i], sum_gamma)
epsilon

array([[[ 0.98805003,  0.40372642],
        [ 0.53797724,  0.23448019],
        [ 0.05338353,  0.51783128]],

       [[ 0.86992223,  0.80669244],
        [ 0.65369358,  0.79343884],
        [ 0.6183715 ,  0.17904411]],

       [[ 0.8651411 ,  0.82390794],
        [ 0.32326851,  0.18333333],
        [ 0.24473372,  0.36866599]],

       [[ 0.60980616,  0.14317828],
        [ 0.82471555,  0.64424388],
        [ 0.43957701,  0.51154068]]])

In [98]:
epsilon.T

array([[[ 0.98805003,  0.86992223,  0.8651411 ,  0.60980616],
        [ 0.53797724,  0.65369358,  0.32326851,  0.82471555],
        [ 0.05338353,  0.6183715 ,  0.24473372,  0.43957701]],

       [[ 0.40372642,  0.80669244,  0.82390794,  0.14317828],
        [ 0.23448019,  0.79343884,  0.18333333,  0.64424388],
        [ 0.51783128,  0.17904411,  0.36866599,  0.51154068]]])

In [100]:
epsilon.T[1][0][:-1]

array([ 0.40372642,  0.80669244,  0.82390794])

In [33]:
print(np.shape(epsilon))
print(np.shape(epsilon.T))

(9, 5, 4)
(4, 5, 9)


In [85]:
alpha = np.zeros( (observation_len, latent_len) )
alpha[1]

array([ 0.,  0.])

In [ ]:
def calc_forward_messages(unary_matrix, 
                          transition_matrix,
                          emission_matrix,
                          latent_indice_len,
                          observation_sentence
                         ):
    """Calcualte the forward messages ~ alpha values.
    
    Input
    -----
    unary_matrix: marginal probabilities ~ initial matrix.
    
    Return
    ------
    alpha
    """
    
    # TODO: verify matrix length
    observation_len = len(observation_sentence)
    
    alpha = np.zeros( (observation_len, latent_indice_len) )
    alpha[0] = np.multiply(emission_matrix[0], unary_matrix)
    
    for t in np.arange(1, observation_len):
        for j in range(latent_indice_len):
            sum_al = 0.0;
#             print("alpha : ", t, j, " :: ", emission_matrix[ observation_sentence[t] ][ j ])
            for i in range(latent_indice_len):
                sum_al += alpha[t-1][i] * transition_matrix[i][j]
#                 print("   sum_al: ", alpha[t-1][i], transition_matrix[i][j])
                
            alpha[t][j] = emission_matrix[ observation_sentence[t] ][ j ] * sum_al
            
    
    return alpha
    
    
def calc_backward_messages(transition_matrix,
                           emission_matrix,
                           latent_indice_len,
                           observation_sentence):
    """Calcualte the backward messages ~ beta values.
    
    Return
    ------
    beta
    """
    # TODO: verify matrix length
    observation_len = len(observation_sentence)
    
    beta = np.zeros( (observation_len, latent_indice_len) )
    beta[-1] = [1]*latent_indice_len
    
    for t in reversed(range(observation_len-1)):
        for i in range(latent_indice_len):
#             print("beta ", t, i)
            for j in range(latent_indice_len):
                beta[t][i] += \
                        beta[t+1][j] \
                        * transition_matrix[i][j] \
                        * emission_matrix[ observation_sentence[t+1] ][j]
#                 print("    ", beta[t+1][j], transition_matrix[i][j], emission_matrix[ observation_sentence[t+1] ][j], beta[t][i])
    
    return beta

def calc_updated_matrix(alpha, beta, latent_indice_len, observation_sentence):
    """Calcualte the gama and epsilon values in order to reproduce better transition and emission matrix.
    
    Return
    ------
    unary_matrix, transition_matrix, emission_matrix
    """
    # TODO: verify matrix length
    observation_len = len(observation_sentence)
    
    
    gamma = np.multiply(alpha, beta)
    epsilon = np.zeros( (observation_len-1, latent_indice_len, latent_indice_len) )
    
    # Normalization on rows
    for i in range(len(gamma)):
        sum_gamma = np.sum(gamma[i])
        gamma[i] = np.divide(gamma[i], sum_gamma)
    
    for t in range(observation_len - 1):   
        for i in range(latent_indice_len):
            for j in range(latent_indice_len):
                epsilon[t][i][j] = 
                        alpha[t][i] * \
                        transition_matrix[j][i] * \
                        beta[t+1][j] * \
                        emission_matrix[ observation_sentence[t+1] ][j]
        # Normalization
        sum_ep = np.sum(epsilon[t])
        epsilon[t] = np.divide(epsilon[t], sum_ep)
    
    print("gamma: ", gamma)
    print("epsilon: ", epsilon)
    
    # Update unary matrix
    new_unary_matrix = np.copy(gamma[0])
    #Normalization unary
    sum_unary = np.sum(new_unary_matrix)
    new_unary_matrix = np.divide(new_unary_matrix, sum_unary)
    
    new_transition_matrix = np.zeros( (latent_indice_len, latent_indice_len) )
    new_emission_matrix = np.zeros( (observation_len, latent_indice_len) )
    
    # Update transition matrix
    for i in range(latent_indice_len):
        sum_gamma = np.sum(gamma.T[i][:-1])
        for j in range(latent_indice_len):
            sum_ep = np.sum( epsilon.T[j][i][:-1] )
            new_transition_matrix[i][j] = sum_ep/sum_gamma
        # Normalization
        sum_trans = np.sum(new_transition_matrix[i])
        new_transition_matrix[i] = np.divide(new_transition_matrix[i], sum_trans)
    
    # Update emission matrix
    sum_gamma = [np.sum(gamma.T[i]) for i in range(latent_indice_len)]   
    for i in range(latent_indice_len):
        new_emission_matrix.T[i] = np.divide(gamma.T[i], sum_gamma[i])
    
    return new_unary_matrix, new_transition_matrix, new_emission_matrix

In [ ]:
unary_matrix = [.85, .15]
transition_matrix = [
    [.3, .7], 
    [.1, .9]
]
emission_matrix = [
    [.4, .5],
    [.6, .5]
]
latent_indice_len = 2
observation_sentence = [0,1,1,0]

alpha = calc_forward_messages(unary_matrix, 
                      transition_matrix,
                      emission_matrix,
                      latent_indice_len,
                      observation_sentence)
print("alpha: ", alpha)
beta = calc_backward_messages(transition_matrix,
                           emission_matrix,
                           latent_indice_len,
                           observation_sentence)
print("beta", beta)
new_unary_matrix, new_transition_matrix, new_emission_matrix = calc_updated_matrix(alpha, beta, latent_indice_len, observation_sentence)
print("New Unary")
print( new_unary_matrix) 
print("New transition")
print(new_transition_matrix) 
print("New emission")
print(new_emission_matrix )

Test

In [230]:
unary_matrix = [.85, .15]
transition_matrix = [
    [.3, .7], 
    [.1, .9]
]
emission_matrix = [
    [.4, .5],
    [.6, .5]
]
latent_indice_len = 2
observation_sentence = [0,1,1,0]

alpha = calc_forward_messages(unary_matrix, 
                      transition_matrix,
                      emission_matrix,
                      latent_indice_len,
                      observation_sentence)
print("alpha: ", alpha)
beta = calc_backward_messages(transition_matrix,
                           emission_matrix,
                           latent_indice_len,
                           observation_sentence)
print("beta", beta)
new_unary_matrix, new_transition_matrix, new_emission_matrix = calc_updated_matrix(alpha, beta, latent_indice_len, observation_sentence)
print("New Unary")
print( new_unary_matrix) 
print("New transition")
print(new_transition_matrix) 
print("New emission")
print(new_emission_matrix )

for i in range(1, 10):
    print (" \n \n Iteration ", i)
    alpha = calc_forward_messages(new_unary_matrix, 
                          new_transition_matrix,
                          new_emission_matrix,
                          latent_indice_len,
                          observation_sentence)
    print("alpha: ", alpha)
    beta = calc_backward_messages(new_transition_matrix,
                               new_emission_matrix,
                               latent_indice_len,
                               observation_sentence)
    print("beta", beta)
    new_unary_matrix, new_transition_matrix, new_emission_matrix = calc_updated_matrix(alpha, beta, latent_indice_len, observation_sentence)
    print("New Unary")
    print( new_unary_matrix) 
    print("New transition")
    print(new_transition_matrix) 
    print("New emission")
    print(new_emission_matrix )


alpha:  [[ 0.34        0.075     ]
 [ 0.0657      0.15275   ]
 [ 0.020991    0.0917325 ]
 [ 0.00618822  0.04862648]]
beta [[ 0.133143  0.127281]
 [ 0.2561    0.2487  ]
 [ 0.47      0.49    ]
 [ 1.        1.      ]]
gamma:  [[ 0.82584825  0.17415175]
 [ 0.30695729  0.69304271]
 [ 0.17998404  0.82001596]
 [ 0.11289345  0.88710655]]
epsilon:  [[[ 0.43103576  0.1162725 ]
  [ 0.22185664  0.23083511]]

 [[ 0.07828248  0.02267046]
  [ 0.4246755   0.47437157]]

 [[ 0.03571255  0.01488023]
  [ 0.3641562   0.58525103]]]
New Unary
[ 0.82584825  0.17415175]
New transition
[[ 0.78566825  0.21433175]
 [ 0.47829664  0.52170336]]
New emission
[[ 0.57926498  0.06764969]
 [ 0.21530542  0.26921421]
 [ 0.12624408  0.31853729]
 [ 0.07918552  0.3445988 ]]
 
 
 Iteration  1
alpha:  [[ 0.47838497  0.01178131]
 [ 0.08213619  0.02925805]
 [ 0.01690704  0.00884864]
 [ 0.01014617  0.00055744]]
beta [[ 0.02180828  0.02299034]
 [ 0.09746168  0.09223037]
 [ 0.46960958  0.31235357]
 [ 1.          1.        ]]
gamma: 

Transition Matrix

In [292]:
def generate_transition_distant_matrix(max_distance, po):
    """ Generate a transition matrix based on jump distance on the latent sentence.
    We extend the latent sentence for 2*length in which each word has 
    an empty word to represent no-alignment state.
    where [max_distance:end] elements are empty words considered as 
    latent words having no direct aligment.
    
    Input
    -----
    max_distance: the length of latent sentence
                  int value
    non_negative_set: random non-negative set as max_distance's size
    po: default value for A->A_empty_word
    
    Output
    ------
    trans_distant_matrix
    """
    
    trans_distant_matrix = np.zeros((2*max_distance, 2*max_distance))
    
    for i in range(max_distance):
        for j in range(max_distance):
            trans_distant_matrix[i][j] = abs(i-j)
    
    print(trans_distant_matrix)
    
    for i in range(max_distance):
        trans_distant_matrix[i+max_distance][i+max_distance] = po
        trans_distant_matrix[i][i+max_distance] = po
        
        sum_d = np.sum(trans_distant_matrix[:max_distance, i])
        trans_distant_matrix[:max_distance, i] = \
                np.divide(
                    trans_distant_matrix[:max_distance, i], 
                    sum_d
                )
        trans_distant_matrix[max_distance:, i] = \
                np.copy(trans_distant_matrix[:max_distance, i])
    
    return trans_distant_matrix

def generate_transition_distant_matrix2(max_distance, non_negative_set, po):
    """ Generate a transition matrix based on jump distance in the latent sentence.
    We extend the latent sentence for 2*length in which each word has 
    an empty word to represent no-alignment state.
    where [max_distance:end] elements are empty words considered as 
    latent words having no direct aligment.
    
    Input
    -----
    max_distance: the length of latent sentence
                  int value
    non_negative_set: random non-negative set as max_distance's size
    po: default value for A->A_empty_word
    
    Output
    ------
    trans_distant_matrix
    """
    
    trans_distant_matrix = np.zeros((2*max_distance, 2*max_distance))
    
    for i in range(max_distance):
        for j in range(max_distance):
            trans_distant_matrix[i][j] = non_negative_set[abs(i-j)]
    
    print(trans_distant_matrix)
    
    for i in range(max_distance):
        trans_distant_matrix[i+max_distance][i+max_distance] = po
        trans_distant_matrix[i][i+max_distance] = po
        
        sum_d = np.sum(trans_distant_matrix[:max_distance, i])
        trans_distant_matrix[:max_distance, i] = \
                np.divide(
                    trans_distant_matrix[:max_distance, i], 
                    sum_d
                )
        trans_distant_matrix[max_distance:, i] = \
                np.copy(trans_distant_matrix[:max_distance, i])
    
    return trans_distant_matrix

Test

In [293]:
print(generate_transition_distant_matrix(3, 0.2))
print(generate_transition_distant_matrix2(3, [5, 7, 6], 0.2))

[[ 0.  1.  2.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.]
 [ 2.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 0.          0.5         0.66666667  0.2         0.          0.        ]
 [ 0.33333333  0.          0.33333333  0.          0.2         0.        ]
 [ 0.66666667  0.5         0.          0.          0.          0.2       ]
 [ 0.          0.5         0.66666667  0.2         0.          0.        ]
 [ 0.33333333  0.          0.33333333  0.          0.2         0.        ]
 [ 0.66666667  0.5         0.          0.          0.          0.2       ]]
[[ 5.  7.  6.  0.  0.  0.]
 [ 7.  5.  7.  0.  0.  0.]
 [ 6.  7.  5.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 0.27777778  0.36842105  0.33333333  0.2         0.          0.        ]
 [ 0.38888889  0.26315789  0.38888889  0.          0.2         0.        ]
 [ 0.33333333  0.36842105  0.27777778  0.          0.          0.2       

In [249]:
import timeit

print(timeit.timeit('import numpy as np; X = np.random.random(15); Y=np.divide(X, np.sum(X))', number=10000))
print(timeit.timeit('import numpy as np; X = np.random.random(15); sum_X = np.sum(X); Y=np.divide(X, sum_X)', number=10000))

0.27222083136439323
0.2220076620578766


# Neural Network

Pre-processing - Word embedding

In [232]:
a = [1,2,3,4]
b = [17,12,11,10]
e = map(lambda x,y:x+y, a,b)

Neural network